This is script will figure out travel times from different runs

In [3]:
import argparse
import pandas

# declare some column names
travelTime = 'travelTime'
startTime = 'startTime'
endTime = 'endTime'
id = 'personId'
tripNumber = 'tripNo'
mode = 'mainMode'

def parse_args(args):
    parser = argparse.ArgumentParser()
    parser.add_argument('-f', '--files', nargs='+', required=True)
    parser.add_argument('-n', '--names', nargs='+', required=True)
    return parser.parse_args(args)

def read_csv(listOfFiles, listOfNames):
    
    assert(len(listOfFiles) == len(listOfNames)), 'number of files and scenario names should be equal but was: (' + str(len(listOfFiles)) + '/' + str(len(listOfNames)) + ')'
    
    result = []

    for file, name in zip(listOfFiles, listOfNames):
        scenario = pandas.read_csv(file, usecols=[id, tripNumber, startTime, endTime, mode])
        scenario.set_index([id, tripNumber], inplace=True)
        scenario.sort_index(inplace=True)
        scenario[travelTime] = scenario[endTime] - scenario[startTime]
        result.append(scenario)
    return result

In [4]:
dataFolder = "C:\\Users\\Janek\\Desktop\\nemo_analysis\\"
args = parse_args([
    '-f', dataFolder + 'base-case.csv', dataFolder + 'sustainable.csv', dataFolder + 'deurbanisation-100.csv', dataFolder + 'smartCity.csv',
    '-n', 'Base Case', 'Gesund & Nachhaltig' , 'Deurbanisation', 'Smart City'
])

dataFrames = read_csv(args.files, args.names)
data = pandas.concat(dataFrames, keys=args.names, names=['RunId', id, tripNumber])
print(data.head(10))

ValueError: Usecols do not match columns, columns expected but not found: ['tripNo', 'personId']

In [ ]:
# select the mode and the travel time
travelTimes = data[[travelTime, mode]]

In [ ]:
# group by mode and by run id
grouped = travelTimes.groupby([mode, 'RunId']).mean()
# unstack the multi-index into two columns for both run ids and then plot
grouped.unstack(level='RunId').plot.bar(title='Travel Times')

In [ ]:
# compute time differences between base and scenario-case

unstacked_travel_times = travelTimes.unstack(level='RunId')
unstacked_travel_times['time difference'] = unstacked_travel_times[travelTime, 'Deurbanisation'] - unstacked_travel_times[travelTime, 'Base Case']

# select trips which have the same mode in both scenarios
is_same_mode = unstacked_travel_times[mode,'Base Case'] == unstacked_travel_times[mode, 'Deurbanisation']
differences_same_mode = unstacked_travel_times[is_same_mode]

#print(differences_same_mode)

baseCaseMainMode = differences_same_mode.xs([(mode, 'Base Case'), ('time difference','')], axis=1)
mean = baseCaseMainMode.groupby((mode, 'Base Case')).mean()
mean.plot.bar(title='Mean trip time difference not changed')

In [ ]:
# select the ones which have different modes in scenario and base case
is_not_same_mode = unstacked_travel_times[mode,'Base Case'] != unstacked_travel_times[mode, 'Deurbanisation']
different_mode = unstacked_travel_times[is_not_same_mode]

#print(different_mode)

baseCaseMainMode = different_mode.xs([(mode, 'Base Case'),(mode, 'Deurbanisation'), ('time difference', '')], axis=1)
mean = baseCaseMainMode.groupby([(mode, 'Base Case'), (mode, 'Deurbanisation')]).mean()
mean.plot.bar(title='time difference, Changer (mean)')